# Chapter 12. Special Methods for Sequences

Tạo 1 class để biểu diễn Vector N chiều
- immutable flat sequence.
- protocols and duck typing.

## Vector: A User-Defined Sequence Type

- Vector will be to use composition, not inheritance
- Store in `array` of float
- Behave like an immutable flat sequence.


## Vector Take #1: Cần tương thích với Vector2d

x, y --> components

In [6]:
# Example 12-2. vector_v1.py: derived from vector2d_v1.py

# tag::VECTOR_V1[]
from array import array
import reprlib
import math


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)  # <1> array: “protected” attribute

    def __iter__(self):
        return iter(self._components)  # <2> unpacking

    def __repr__(self):
        components = reprlib.repr(self._components)  # <3> get a limited-length representation (VD: Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...]) )
        components = components[components.find('['):-1]  # <4> Remove the array('d', prefix, and the trailing )
        # Vector(array('d', [3.0, 4.0, 5.0])) --> Vector([3.0, 4.0, 5.0])
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))  # <5>

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(*self)  # <6>

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)  # <7> Pass memoryview trực tiếp từ constructor, mà không cần unpacking (*memv)
# end::VECTOR_V1[]

In [7]:
Vector(range(10))

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

`__str__`, `__eq__`, and `__bool__` methods are unchanged from Vector2d

## Protocols and Duck Typing

- don’t need to inherit from any special class to create a __fully functional sequence type__ in Python
- In the context of object-oriented programming, a protocol is an informal interface
    - defined only in documentation and not in code.

In [8]:
# Example 12-3. Code from Example 1-1, reproduced here for convenience

import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                       for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

Class trên implement sequence protocol `__len__` và `__getitem__`
--> Coder có thể hiểu sequence của nó
--> Nó được biết là *duck typing*

## Vector Take #2: A Sliceable Sequence

Hỗ trợ sequence protocol sẽ rất dễ nếu định nghĩa được sequence attribute trong object (`self._components` array)
--> Dễ dàng impl `__len__` và `__getitem__`

In [9]:
from array import array
import reprlib
import math
import operator


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    # tag::VECTOR_V2[]
    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):  # <1>
            cls = type(self)  # <2>
            return cls(self._components[key])  # <3>
        index = operator.index(key)  # <4>
        return self._components[index]  # <5> Nếu không có 1,2,3,4 , nó sẽ return array('d', [1.0, 2.0, 3.0]) thay vì Vector([1.0, 2.0, 3.0])

    # end::VECTOR_V2[]

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

# 1: If the key argument is a slice (v7[1:4])
# 2: …get the class of the instance (i.e., Vector) and…
# 3: …invoke the class to build another Vector instance from a slice of the _components array.

In [20]:
v7 = Vector(range(7))
len(v7)

7

In [24]:
v7[1:4]

Vector([1.0, 2.0, 3.0])

In [28]:
v7[1:7:2]

Vector([1.0, 3.0, 5.0])

In [25]:
v7[2]

2.0

In [30]:
v7[-1:]

Vector([6.0])

### How Slicing (':') Works

In [16]:
class MySeq:
    def __getitem__(self, index):
        return index

s = MySeq()
s[1]

1

In [17]:
s[1:4:2, 9] # means start at 1, stop at 4, step by 2

(slice(1, 4, 2), 9)

In [18]:
s[1:4:2, 7:9]

(slice(1, 4, 2), slice(7, 9, None))

Khi slice, nó return tuple, tuple nó hold các `slide` object

In [29]:
dir(slice)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'indices',
 'start',
 'step',
 'stop']

## Vector Take #3: Dynamic Attribute Access

It may be convenient to access the first few components with shortcut letters such as x, y, z instead of v[0], v[1], and v[2]

In [42]:
# Example 12-8: __getattr__ checks whether the attribute being sought is one of the letters xyzt
# if so, returns the corresponding vector component.
from array import array
import reprlib
import math
import operator


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        index = operator.index(key)
        return self._components[index]

    # tag::VECTOR_V3_GETATTR[]
    __match_args__ = ('x', 'y', 'z', 't')  # <1>

    def __getattr__(self, name):
        cls = type(self)  # <2>
        try:
            pos = cls.__match_args__.index(name)  # <3>
        except ValueError:  # <4>
            pos = -1
        if 0 <= pos < len(self._components):  # <5>
            return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}'  # <6>
        raise AttributeError(msg)
    # end::VECTOR_V3_GETATTR[]

    # tag::VECTOR_V3_SETATTR[]
    # Nếu ko có nó, v.x = 10 có thể được set -> mất đi immutable
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:  # <7>
            if name in cls.__match_args__:  # <8>
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():  # <9>
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = 'aaa'  # <10>
            if error:  # <11>
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)  # <6>

    # end::VECTOR_V3_SETATTR[]

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

# 1: Set __match_args__ to allow positional pattern matching on the dynamic attributes supported by __getattr__

# 2: Get the Vector class for later use.

# 7: Special handling for single-character attribute names.

# 11: If there is a nonblank error message, raise AttributeError

# 12: Default case: call __setattr__ on superclass for standard behavior.

`super()` function provides a way to access methods of superclasses dynamically,
- Rất thích hợp với dynamic language hỗ trợ multiple inheritance (VD: Python).

In [43]:
v = Vector(range(5))
v.y, v[1]

(1.0, 1.0)

In [44]:
v.k

AttributeError: 'Vector' object has no attribute 'k'

In [45]:
v.y = 10

AttributeError: readonly attribute 'y'

In [46]:
v.k = 10

AttributeError: can't set attributes 'a' to 'z' in 'Vector'

In [47]:
v.Y = 10

AttributeError: aaa

In [48]:
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

WARNING

Việc khai báo `__slots__` ở cấp độ lớp sẽ ngăn việc setting new instance attributes, bạn nên sử dụng tính năng đó thay vì triển khai `__setattr__` như đã làm.

Tuy nhiên, có vài vấn đề với `__slots__` ta đã nói ở chương trước
- Không nên dùng nó chỉ cho việc ngăn việc set attr.
- Chỉ nên dùng để save memory.

--> Khi viết `__getattr__`, cũng cần viết `__setattr__`

`Vector` will remain __immutable__ -> make it __hashable__.

## Vector Take #4: Hashing and a Faster ==

```Python
# Hash in Vector2D
def __hash__(self):
        return hash((self.x, self.y)) # computed the hash of a tuple

v1 = Vector2d(3, 4)
hash(v1)
1079245023883434373
```

Now, deal với hàng nghìn attr , thì dùng tuple như 2D rất tốn kém
--> dùng XOR (`^`)
--> v[0] ^ v[1] ^ v[2] ^ v[3]
--> dùng `functools.reduce`
- Reducing func (sum, any, all) tạo ra một kết quả tổng hợp duy nhất từ ​​một chuỗi

In [49]:
# functools.reduce using an anonymous function.
import functools
functools.reduce(lambda a, b: a^b, range(6))

1

In [50]:
import operator
functools.reduce(operator.xor, range(6))

1

In [ ]:
# __eq__ and __hash__

from array import array
import reprlib
import math
import functools
import operator


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other))) # <3>

    def __hash__(self):
        hashes = (hash(x) for x in self) # <4>
        # Hoặc có thể dùng map để save mem (vì nó là lazy)
        # hashes = map(hash, self._components)
        return functools.reduce(operator.xor, hashes, 0) # <5>

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        index = operator.index(key)
        return self._components[index]

    __match_args__ = ('x', 'y', 'z', 't')

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components):
            return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}'
        raise AttributeError(msg)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

# 3: zip produces a generator of tuples
# 4: Create a generator expression to lazily compute the hash of each component.
# 5: initializer = 0

WARNING

initializer:
- is the value returned if the sequence is empty
- is used as the first argument in the reducing loop
    - 0 for +, |, ^
    - 1 for *, &

In [58]:
# zip: như cái khoá kéo
zip(range(3), 'ABC', 'abs')
list(zip(range(3), 'ABC', 'abcd'))

[(0, 'A', 'a'), (1, 'B', 'b'), (2, 'C', 'c')]

In [59]:
from itertools import zip_longest
list(zip_longest(range(3), 'ABC', 'abcd'))

[(0, 'A', 'a'), (1, 'B', 'b'), (2, 'C', 'c'), (None, None, 'd')]

zip còn được dùng để đảo ma trận

In [60]:
b = [(1, 2),
     (3, 4),
     (5, 6)]
list(zip(*b))

[(1, 3, 5), (2, 4, 6)]

## Vector Take #5: Formatting

In [61]:
from array import array
import reprlib
import math
import functools
import operator
import itertools  # <1>


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector({components})'

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.hypot(*self)

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        index = operator.index(key)
        return self._components[index]

    __match_args__ = ('x', 'y', 'z', 't')

    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        if 0 <= pos < len(self._components):
            return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}'
        raise AttributeError(msg)

    def angle(self, n):  # <2>
        r = math.hypot(*self[n:])
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):  # <3>
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())  # <4>
            outer_fmt = '<{}>'  # <5>
        else:
            coords = self
            outer_fmt = '({})'  # <6>
        components = (format(c, fmt_spec) for c in coords)  # <7>
        return outer_fmt.format(', '.join(components))  # <8>

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
# end::VECTOR_V5[]

# 2: Tính toán một trong các tọa độ góc
# 3: Generator expression để tính all các toạ độ góc
# 4: Sử dụng itertools.chain để tạo genexp để lặp lại liên tục trên độ lớn và tọa độ góc.

`Vector` object in 4D space `(len(v) == 4)`, the 'h' code will produce a display like `<r, Φ₁, Φ₂, Φ₃>`

In [62]:
format(Vector([-1, -1, -1, -1]), 'h')

'<2.0, 2.0943951023931957, 2.186276035465284, 3.9269908169872414>'

In [63]:
format(Vector([-1, -1, -1, -1]), '.2f')

'(-1.00, -1.00, -1.00, -1.00)'

In [64]:
v1 = Vector([3, 4, 5])
bytes(v1)

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x14@'

In [65]:
v1_clone = Vector.frombytes(bytes(v1))
v1_clone

Vector([3.0, 4.0, 5.0])

In [66]:
v1 == v1_clone

True